In [58]:
import numpy as np
import pandas as pd
import random as rd

Structure of overall applications.

Get data ready and split into two groups. Group 1 is prop on the impromptu motion on Day 1 and Group 2 is opp on the prepped motion of Day 1. On Day 2, Group 1 is opp on the impromptu and Group 2 is Prop on the impromptu. The alternation continues on Day 3 and Day 4.

Then the same process is applied for each day. First, pairs are formed between countries within groups. The pairs from both groups get paired into 2x2 blocks. 

Day 1 to 4, with the sides that groups take alternating
Pairs (for both groups)
Blocks (update penalty scores)


In [59]:
rankings = pd.read_csv('WSDCrankings.csv', sep=';')
rankings['penalty'] = pd.Series(np.repeat(0,62), index=rankings.index) # Create penalty column

In [60]:
rankings.count()

nation      62
strength    62
penalty     62
dtype: int64

In [61]:
countries = rankings['nation'].as_matrix() # Save countries as array for use later.

In [62]:
group1 = countries[range(1,61,2)]
group2 = countries[range(2,62,2)]

In [63]:
group1

array(['South Africa', 'England', 'Ireland', 'Greece', 'Malaysia',
       'Netherlands', 'Peru', 'United Arab Emirates', 'Croatia', 'Estonia',
       'Scotland', 'United States', 'Philippines', 'Denmark', 'China',
       'Israel', 'Czech Republic', 'Slovakia', 'Argentina', 'Chile',
       'Turkey', 'Lithuania', 'Sweden', 'Taiwan', 'Macau', 'Bahrain',
       'Kuwait', 'Japan', 'Nigeria', 'Rwanda'], dtype=object)

In [64]:
averageStrength = rankings['strength'].mean()
averageStrength

0.5682548387096773

In [65]:
rankings.loc[rankings['nation'].isin(group1)] # Select group of countries to start making pairs.

,nation,strength,penalty
1,South Africa,0.0859,0
3,England,0.1166,0
5,Ireland,0.1369,0
7,Greece,0.2196,0
9,Malaysia,0.2307,0
11,Netherlands,0.2769,0
13,Peru,0.2935,0
15,United Arab Emirates,0.3881,0
17,Croatia,0.3962,0
19,Estonia,0.4125,0


In [66]:
def generatePairs():

    # Pair countries based on two criteria: strength and penalty. 
    # Iterate through countries in the group.
    # Take the sum of the difference between the strength and penalty for each combination.
    # Choose partner with lowest sum.
    # Put paired countries on list that are no longer available for partnering.
    # Repeat.
    
    # Create a table to store all the matches in.
    pairTable = pd.DataFrame(columns=('nation', 'pairno','groupno'))
    pairCount = 0
    
    # If the group I am matching is group 2, I remove group 1 countries.
    for group in [group1, group2]:
        if group is group2: 
            pairCount += 100
            groupno = 2
            # Below does the same as a for loop but is more efficient.
            countrieslist = [x for x in countries.tolist() if x not in group1]

        else:
            groupno = 1
            # Same as above
            countrieslist = [x for x in countries.tolist() if x not in group2]


        for countryOne in group:

            if countryOne in countrieslist: # If country was already matched, can't match again.
                # Remove now, so it is not matched with self and is not matched again.
                countrieslist.remove(countryOne) 
                pairCount += 1 # Get ready to match a pair with this number

                # Following the logic that the Average strength of a pair is the pair's scores divided by two we can find the 
                # country that brings the pair closest to the average.
                bestMatchStrength = 2 * averageStrength - rankings['strength'].loc[rankings['nation'] == countryOne].values[0]

                # Filter the table for countries still requiring a matching
                matchableCountries = rankings.loc[rankings['nation'].isin(countrieslist)]

                bestMatchedStrength = min(matchableCountries['strength'].as_matrix()
                                          , key=lambda x:abs(x-bestMatchStrength))
                bestMatchedCountries = matchableCountries['nation'].loc[matchableCountries['strength'] == bestMatchedStrength].values
                bestMatchedCountry = bestMatchedCountries[rd.randint(0,len(bestMatchedCountries) - 1)]
                countrieslist.remove(bestMatchedCountry)

                # Powerdif is simply the absolute difference between the Strength of the two countries in the pair.
                powerdif = abs(rankings['strength'].loc[rankings['nation'] == countryOne].values[0] -
                               rankings['strength'].loc[rankings['nation'] == bestMatchedCountry].values[0])

                newPairing = pd.DataFrame({'nation': [countryOne, bestMatchedCountry],
                                           'pairno': [pairCount, pairCount],
                                           'groupno': [groupno, groupno],
                                           'powerdif': [powerdif, powerdif]})
                pairTable = pd.concat([pairTable, newPairing])
    
    return(pairTable)

In [67]:
def generateAttributes():
    attributesTable = pd.DataFrame(columns=('pairno', 'strength','penalty', 'groupno'))
    pairNumbers = np.unique(pairTable['pairno'].values)
    
    for pairNumber in pairNumbers:
        pairCountries = pairTable['nation'].loc[pairTable['pairno'] == pairNumber].values
        meanStrength = np.mean(rankings['strength'].loc[rankings['nation'].isin(pairCountries)])
        meanPenalty = np.mean(rankings['penalty'].loc[rankings['nation'].isin(pairCountries)])
        groupNo = pairTable['groupno'].loc[pairTable['pairno'] == pairNumber].values[0]
        powerdif = pairTable['powerdif'].loc[pairTable['pairno'] == pairNumber].values[0]
        
        pairAttributes = pd.DataFrame({'pairno': [pairNumber],
                                       'strength': [meanStrength],
                                       'powerdif': [powerdif],
                                       'groupno': [groupNo],
                                       'penalty': [meanPenalty],
                                       'matchScore':[.5*powerdif + .5*meanStrength]})
        attributesTable = pd.concat([attributesTable, pairAttributes])
    
    return(attributesTable)

In [89]:
def generateBlocks(pairTable):
    
    # Let's make some blocks!
    pairNumbers = np.unique(pairTable['pairno'].values)
    unmatchedPairs = pairNumbers.tolist() # List of all pair numbers, because all remain to be matched.
    blockTable = pd.DataFrame(columns=('pairno', 'blockno','penalty'))
    blockCount = 0
    
    for pairNumber in pairNumbers:
        
        if pairNumber in unmatchedPairs:

            # Get data necessary to sort legal matches.
            groupNo = attributesTable['groupno'].loc[attributesTable['pairno'] == pairNumber].values[0]
            unmatchedPairs.remove(pairNumber) # Remove so pair doesn't match itself.
            blockCount += 1
            
            # Statement checks if the group is from the other group and filters out already matched pairs.
            matchablePairs = attributesTable.loc[(attributesTable['groupno'] != groupNo) & 
                                                 (attributesTable['pairno'].isin(unmatchedPairs))]
            
            pairMatchScore = attributesTable['matchScore'].loc[attributesTable['pairno'] == pairNumber].values[0]
            pairpenalty = attributesTable['penalty'].loc[attributesTable['pairno'] == pairNumber].values[0]
            
            bestMatchedScore = min(matchablePairs[['matchScore','penalty']].as_matrix(),
                                   key = lambda x: abs((x[0]+x[1])-(pairMatchScore+pairpenalty)))[0]
            bestMatchedPair = matchablePairs['pairno'].loc[matchablePairs['matchScore'] == bestMatchedScore].values[0]
            unmatchedPairs.remove(bestMatchedPair)
            
            penaltyPair = pairpenalty + (bestMatchedScore - averageMatchScore)
            penaltyMatchedPair = matchablePairs['penalty'].loc[matchablePairs['matchScore'] == bestMatchedScore].values[0] + (pairMatchScore - averageMatchScore) 
            
            blockTable = pd.concat([blockTable, 
                                    pd.DataFrame({'pairno': [pairNumber, bestMatchedPair],
                                                  'blockno': [blockCount, blockCount],
                                                  'penalty': [penaltyPair, penaltyMatchedPair]})])
    
    return blockTable

In [69]:
def generateDraw(blockTable):
    
    # The end result of this function is two tables: round 1 and round 2
    # Group 1 countries are prop in the first round
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-69-8e1c0e567b89>, line 6)

In [70]:
def updatePenalties(blockTable):
    
    # Don't forget to sort by penalty at the end!

SyntaxError: unexpected EOF while parsing (<ipython-input-70-ce20a3913fd6>, line 3)

In [91]:
pairTable = generatePairs()
attributesTable = generateAttributes()
averageMatchScore = attributesTable['matchScore'].mean()

In [92]:
blockTable = generateBlocks(pairTable)
blockTable

,blockno,pairno,penalty
0,1.0,1.0,0.192052
1,1.0,117.0,0.194452
0,2.0,2.0,0.183427
1,2.0,118.0,0.186777
0,3.0,3.0,0.198702
1,3.0,132.0,0.171952
0,4.0,4.0,0.119702
1,4.0,119.0,0.109652
0,5.0,5.0,0.088077
1,5.0,120.0,0.079127


In [79]:
generateDraw(blockTable)
updatePenalties(blockTable)

NameError: name 'generateDraw' is not defined

In [97]:
blockTable.sort_values('penalty')

,blockno,pairno,penalty
1,16.0,115.0,-0.119555
1,14.0,101.0,-0.035855
0,13.0,13.0,-0.033805
1,13.0,116.0,-0.027955
0,1.0,1.0,-0.026405
1,1.0,113.0,-0.025305
0,14.0,14.0,-0.020505
0,11.0,11.0,-0.017605
1,11.0,112.0,-0.012655
1,2.0,105.0,-0.009955
